# [rinna](https://huggingface.co/rinna)

## 下準備

In [8]:
!pip install --upgrade pip
!pip install torch
!pip install sentencepiece # [ubuntu にsentencepiece をインストール](https://qiita.com/smoto-shei/items/81f462d624c8fceea8e4)
!pip install transformers
!pip install torchinfo

!pip install -U transformers
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U accelerate

  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/1e/2c/af22cd797fc368a9f098ed03015730e6568b884fe67f9940793d944a4b7b/bitsandbytes-0.41.1-py3-none-any.whl.metadata
Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.0
    Uninstalling bitsandbytes-0.37.0:
      Successfully uninstalled bitsandbytes-0.37.0
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/4d/a7/05c67003d659a0035f2b3a8cf389c1d9645865aee84a73ce99ddab16682f/accelerate-0.22.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [9]:
!echo "avionics" | sudo -S apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

パッケージリストを読み込んでいます... 完了0%
依存関係ツリーを作成しています... 完了%         
状態情報を読み取っています... 完了          
build-essential はすでに最新バージョン (12.9ubuntu3) です。
libgoogle-perftools-dev はすでに最新バージョン (2.9.1-0ubuntu3) です。
pkg-config はすでに最新バージョン (0.29.2-1ubuntu3) です。
cmake はすでに最新バージョン (3.22.1-1ubuntu1.22.04.1) です。
アップグレード: 0 個、新規インストール: 0 個、削除: 0 個、保留: 2 個。


## ファインチューニング
 - 参考 [自然言語処理初心者が「GPT2-japanese」で遊んでみた](https://developer.mamezou-tech.com/blogs/2022/07/08/gpt-2-japanese/)
 - CUDA 12.1以上だとなんかエラー出るっぽい。（推奨は12.0.1）
 - transformers, bitsandbytes, peft, accelerateあたりを最新にしたほうが良さそう。（互換性はあるのだが、よくわからない。）

In [10]:
!pip install git+https://github.com/huggingface/transformers
!git clone https://github.com/huggingface/transformers
!pip install -r ./transformers/examples/pytorch/language-modeling/requirements.txt

# !pip install accelerate==0.20.3 #0.20.3以上じゃないとファインチューニングでエラーが出る

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vhcssa_p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vhcssa_p
  Resolved https://github.com/huggingface/transformers to commit ef10dbce5cbc9a8b6a0a90b04378ca96f4023aa1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'transformers' already exists and is not an empty directory.


In [16]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt2-small \
  --train_file=data/baki_10M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

09/01/2023 13:52:30 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
09/01/2023 13:52:30 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_trans

### 入力文含むプロンプトの設定

In [21]:
prompt = [
    # {
    #     "speaker": "ユーザー",
    #     "text": "山地において敵を挟撃するためにどのような戦術が有効ですか？"
    # },
    {
        "speaker": "ユーザー",
        "text": "地上最強の生物にふさわしい男は誰ですか？"
    },
    # {
    #     "speaker": "ユーザー",
    #     "text": "君たちはどう生きるか"
    # }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)
print(prompt)

ユーザー: 地上最強の生物にふさわしい男は誰ですか？<NL>システム: 


In [22]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import torch

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32000, bias=False)
)

### 実行

In [23]:
%%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
# %%timeit

# import time

# # 時間計測開始
# time_sta = time.time()

# 生成する文章の数
num = 1

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=10, # 最短の文章長
        do_sample=True,
        top_k=100, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

# # 時間計測終了
# time_end = time.time()
# # 経過時間（秒）
# tim = time_end- time_sta
# print(tim)

ユーザー: 地上最強の生物にふさわしい男は誰ですか?<nl>システム: 俺の身体がそれに耐えられず半年後に死ぬというのならそれだけの男だったということだろう
CPU times: user 207 ms, sys: 0 ns, total: 207 ms
Wall time: 207 ms


## ファインチューニングなし

In [ ]:
%%time
%%timeit

import torch
from transformers import AutoTokenizer, T5Tokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-small", use_fast=False)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading for rinna/japanese-gpt2-xsmall

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")

# GPU モデルが軽ければ動く(mediumくらいまで？)
if torch.cuda.is_available():
    model = model.to("cuda")

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to("cuda")

# 生成する文章の数
num = 1

with torch.no_grad():
    output = model.generate(
        input_ids,
        do_sample=True,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]], #これがあるとエラー出る
        num_return_sequences=num # 生成する文章の数
    )

decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
    print(decoded[i])

## [rinna/japanese-gpt-neox-3.6b-instruction-sft](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft)

In [ ]:
%%time
# %%timeit

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft")

# # GPU (VRAM:8GBだと動かない)
# if torch.cuda.is_available():
#     model = model.to("cuda")

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.001,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
output = output.replace("<NL>", "\n")
print(output)
# """分かりました。いくつかのおすすめを紹介します。
# 1. ハチ公像です。ハチ公像は、日本の観光スポットの1つとして人気があります。
# 2. スクランブル交差点です。多くの人々が行き交う大きな交差点で、観光客に人気のスポットです。
# 3. 109です。109は、ショッピングやエンターテイメント施設です。
# 4. 道玄坂です。道玄坂は、日本の商業地区である坂道です。</s>"""

## [rinna/japanese-gpt-neox-3.6b-instruction-sft-v2](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft-v2)

In [ ]:
%%time

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft-v2", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft-v2")

# # GPU (VRAM:8GBだと動かない)
# if torch.cuda.is_available():
#     model = model.to("cuda")

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.001,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
output = output.replace("<NL>", "\n")
print(output)
"""わかりました。まずは、コンタクトレンズを長時間着用することによる目の乾燥を防ぐことができます。また、毎日同じ時間帯にコンタクトレンズを着用してみることもできます。そして、コンタクトレンズが目に合わないような場合は、新しいものを試してみる必要があります。</s>"""
